# It's a Machine and Natural Language Model

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_ud, load_wikitext
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re

seed = 691

print("Loading pre-training data...")
pretrain_path = '/data/newstweet/week_2019-40_article_texts/'
total_pretrain = len([pretrain_file for pretrain_file in os.listdir(pretrain_path) if re.search("^\d+.txt$", pretrain_file)])
num_pretrain = 0 # total_pretrain

all_pretrain_files = [pretrain_file for pretrain_file in os.listdir(pretrain_path) if re.search("^\d+.txt$", pretrain_file)]
if num_pretrain:
    np.random.seed(seed)
    pretrain_files = np.random.choice(all_pretrain_files, size=num_pretrain, replace=False)
else:
    pretrain_files = np.array([])

ptdocs = [[[open(pretrain_path+pretrain_file).read()]] for pretrain_file in tqdm(pretrain_files)]

max_char = 200_000_000
m = 10; space = False; fine_tune = False; fine_tune_post_pretrain = False
positionally_encode = 'c'; do_ife = True; update_ife = False; update_bow = False
runners = 0; gpu = False
## 
print("Loading WikiText LM'ing data...")
load_set = '2'; tokenizer = 'hr-bpe'
train_docs = load_wikitext(v = load_set, split = "train", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = load_wikitext(v = load_set, split = "test", num_articles = 0, seed = seed, space = space, rebuild = True)
test_docs = [doc for doc in test_docs if len(doc['text']) <= max_char][:1]
train_docs = [doc for doc in train_docs if len(doc['text']) <= max_char][:4]
##

##
# print("Loading gold-tagged UDs data...")
# load_set = 'GUM'; tokenizer = 'hr-bpe'
# all_docs = load_ud("English", num_articles = 0, seed = 691, load_set = load_set, rebuild = True, space = space)
# test_docs = [doc for doc in all_docs if 'test' in doc['id'] and len(doc['text']) <= max_char][:2]
# train_docs = [doc for doc in all_docs if 'test' not in doc['id'] and len(doc['text']) <= max_char][:10]
##
nsamp = len(test_docs)
print('Avail. pre-train, total pre-train, Avail. gold, total gold-train, total test-gold: ', 
      total_pretrain, len(ptdocs), len(train_docs) + len(test_docs), len(train_docs), len(test_docs))

Loading pre-training data...


0it [00:00, ?it/s]

Loading WikiText LM'ing data...


Avail. pre-train, total pre-train, Avail. gold, total gold-train, total test-gold:  14198 0 5 4 1


In [2]:
##
docs = [["".join(s) for s in d['document']] for d in train_docs]
tdocs = [["".join(s) for s in d['document']] for d in test_docs]
covering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in train_docs]
tcovering = [[re.split('( )', ''.join(s)) for s in d['document']] for d in test_docs]
covering_vocab = set([t for d in covering for s in d for t in s])
all_layers = {}
##

##
# docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
# tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
# covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
# tcovering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs]
# covering_vocab = set([t for d in covering for s in d for t in s])
# all_layers = {}
# all_layers = {d_i: {# 'lem': [[row[2] for row in s] for s in d['conllu']], 
#                     'pos': [[row[3] for row in s] for s in d['conllu']], 
#                     'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
#                     'dep': [[row[7] for row in s] for s in d['conllu']],
#                     'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])]}
#               for d_i, d in enumerate(train_docs)}
##

model = LM(m = m, tokenizer = tokenizer, noise = 0.001, seed = seed, space = space,
           positionally_encode = positionally_encode, do_ife = do_ife, 
           runners = runners, gpu = gpu)
model.fit(docs, f'{load_set}-{nsamp}', covering = covering, fine_tune = fine_tune, all_layers = all_layers) # 
model.pre_train(ptdocs, update_ife = update_ife, update_bow = update_bow)
if fine_tune_post_pretrain:
    model.fine_tune(docs, covering = covering)

Training tokenizer...


Fitting:   7%|▋         | 7/100 [00:02<00:26,  3.46it/s]


Built a vocabulary of 2196 types
Tokenizing documents...


100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


Counting documents...


...

Counting for transition matrices...


100%|██████████| 4/4 [00:00<00:00, 27.01it/s]


Building transition matrices for Viterbi tag decoding...


100%|██████████| 6/6 [00:00<00:00, 9903.91it/s]

Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 60218 2317 99 2079 111 305 1.25


```
--- wt2 (4/1),ife,0-pt,P1
100%|██████████| 1201/1201 [00:08<00:00, 141.79it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 1235.43 1235.43 1201 1201
--- wt2 (4/1),ife,0-pt,P2
 100%|██████████| 1201/1201 [00:09<00:00, 125.11it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 806.09 806.09 1201 1201
--- wt2 (10/2),ife,0-pt,P1
100%|██████████| 1083/1083 [00:11<00:00, 98.32it/s] 
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 2045.8 2045.8 1083 1083
--- wt2 (10/2),ife,0-pt,P2
100%|██████████| 1083/1083 [00:11<00:00, 91.07it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 50.0 1297.37 1297.37 1083 1083
100%|██████████| 6406/6406 [01:11<00:00, 90.04it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 100.0 1386.57 1386.57 6406 6406
--- wt2 (100/2),ife,0-pt,P1
100%|██████████| 1014/1014 [03:49<00:00,  4.42it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 519.85 519.85 1014 1014
--- wt2 (100/2),ife,0-pt,P2
100%|██████████| 1014/1014 [03:56<00:00,  4.28it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 5.0 410.7 410.7 1014 1014
100%|██████████| 6128/6128 [24:18<00:00,  4.20it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 10.0 534.03 534.03 6128 6128
```

In [3]:
model.generate(docs = tdocs[:1], m = 50, seed = seed)

Tokenizing documents...


100%|██████████| 1/1 [00:00<00:00,  7.63it/s]


Evaluating language model..


100%|██████████| 1201/1201 [00:08<00:00, 142.01it/s]

document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 1496.16 1496.16 1201 1201


```
--- gum (10/2),ife,0-pt,P1
100%|██████████| 2217/2217 [00:15<00:00, 142.51it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 2092.63 2092.63 2217 2217
--- gum (10/2),ife,0-pt,P2
100%|██████████| 2217/2217 [00:18<00:00, 123.10it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 50.0 923.51 923.51 2217 2217
100%|██████████| 1965/1965 [00:15<00:00, 123.25it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 100.0 1662.75 1662.75 1965 1965
--- gum (10/2),ife,100-pt (no bow update),P1
100%|██████████| 2217/2217 [00:16<00:00, 136.28it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 2251.24 2251.24 2217 2217
--- gum (10/2),ife,100-pt (no bow update),P2
100%|██████████| 2217/2217 [00:18<00:00, 121.18it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 50.0 1143.31 1143.31 2217 2217
100%|██████████| 1965/1965 [00:16<00:00, 118.54it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 100.0 2525.91 2525.91 1965 1965
--- gum (10/2),ife,100-pt (yes bow updated),P1
100%|██████████| 2217/2217 [00:16<00:00, 135.62it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 2422.28 2422.28 2217 2217
--- gum (10/2),ife,100-pt (yes bow updated),P2
100%|██████████| 2217/2217 [00:18<00:00, 118.71it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 50.0 1147.52 1147.52 2217 2217
100%|██████████| 1965/1965 [00:17<00:00, 114.86it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 100.0 2537.5 2537.5 1965 1965
```

In [4]:
model.fine_tune(docs, covering = covering)
model.generate(docs = tdocs, m = 50, seed = seed)

Fine-tuning dense output heads...


  0%|          | 0/4 [00:02<?, ?it/s]


AttributeError: 'LM' object has no attribute '_positional_intensities'

```
--- gum (132/18),ife,0-pt,P1
100%|██████████| 1781/1781 [03:30<00:00,  8.47it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 1485.29 1485.29 1781 1781
--- gum (132/18),ife,0-pt,P2
100%|██████████| 1781/1781 [04:05<00:00,  7.27it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 6.0 460.62 460.62 1781 1781
100%|██████████| 1767/1767 [04:03<00:00,  7.26it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 11.0 1167.17 1167.17 1767 1767
100%|██████████| 1279/1279 [02:56<00:00,  7.27it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 17.0 1054.41 1054.41 1279 1279
100%|██████████| 1747/1747 [04:00<00:00,  7.25it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 22.0 475.11 475.11 1747 1747
100%|██████████| 1352/1352 [03:18<00:00,  6.80it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 28.000000000000004 265.98 265.98 1352 1352
100%|██████████| 1807/1807 [04:07<00:00,  7.29it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 33.0 487.56 487.56 1807 1807
100%|██████████| 2012/2012 [04:35<00:00,  7.29it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  7 39.0 249.98 249.98 2012 2012
100%|██████████| 1383/1383 [03:09<00:00,  7.29it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  8 44.0 477.93 477.93 1383 1383
100%|██████████| 1565/1565 [03:33<00:00,  7.33it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  9 50.0 592.21 592.21 1565 1565
100%|██████████| 2370/2370 [05:35<00:00,  7.07it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  10 56.00000000000001 515.75 515.75 2370 2370
100%|██████████| 1124/1124 [02:32<00:00,  7.36it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  11 61.0 560.09 560.09 1124 1124
100%|██████████| 1745/1745 [03:58<00:00,  7.31it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  12 67.0 349.67 349.67 1745 1745
100%|██████████| 1910/1910 [04:19<00:00,  7.35it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  13 72.0 334.99 334.99 1910 1910
100%|██████████| 1374/1374 [03:18<00:00,  6.92it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  14 78.0 293.69 293.69 1374 1374
100%|██████████| 1989/1989 [04:33<00:00,  7.26it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  15 83.0 342.48 342.48 1989 1989
100%|██████████| 1239/1239 [02:48<00:00,  7.35it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  16 89.0 345.03 345.03 1239 1239
100%|██████████| 1112/1112 [02:32<00:00,  7.30it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  17 94.0 378.08 378.08 1112 1112
100%|██████████| 1869/1869 [04:14<00:00,  7.34it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  18 100.0 257.75 257.75 1869 1869
--- gum (132/18),ife,all-pt,P1
100%|██████████| 1781/1781 [11:03<00:00,  2.68it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 632.12 632.12 1781 1781
--- gum (132/18),ife,all-pt,P1 (prose = 0.75, style = 0.25, chunk = 0.9, punct = 0.999)
100%|██████████| 1781/1781 [14:44<00:00,  2.01it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 547.11 547.11 1781 1781
--- gum (132/18),ife,all-pt,P2
100%|██████████| 1781/1781 [11:44<00:00,  2.53it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 6.0 605.41 605.41 1781 1781
100%|██████████| 1767/1767 [11:51<00:00,  2.48it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 11.0 2030.26 2030.26 1767 1767
100%|██████████| 1279/1279 [08:28<00:00,  2.51it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 17.0 1967.31 1967.31 1279 1279
100%|██████████| 1747/1747 [11:29<00:00,  2.53it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 22.0 724.38 724.38 1747 1747
100%|██████████| 1352/1352 [08:47<00:00,  2.56it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 28.000000000000004 461.66 461.66 1352 1352
100%|██████████| 1807/1807 [11:57<00:00,  2.52it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  6 33.0 767.16 767.16 1807 1807
100%|██████████| 2012/2012 [13:24<00:00,  2.50it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  7 39.0 316.76 316.76 2012 2012
100%|██████████| 1383/1383 [09:01<00:00,  2.55it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  8 44.0 664.81 664.81 1383 1383
100%|██████████| 1565/1565 [10:18<00:00,  2.53it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  9 50.0 851.8 851.8 1565 1565
100%|██████████| 2370/2370 [15:26<00:00,  2.56it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  10 56.00000000000001 643.37 643.37 2370 2370
100%|██████████| 1124/1124 [07:33<00:00,  2.48it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  11 61.0 803.8 803.8 1124 1124
100%|██████████| 1745/1745 [11:21<00:00,  2.56it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  12 67.0 425.11 425.11 1745 1745
100%|██████████| 1910/1910 [12:41<00:00,  2.51it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  13 72.0 477.82 477.82 1910 1910
100%|██████████| 1374/1374 [09:17<00:00,  2.46it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  14 78.0 379.09 379.09 1374 1374
100%|██████████| 1989/1989 [12:56<00:00,  2.56it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  15 83.0 441.64 441.64 1989 1989
100%|██████████| 1239/1239 [08:21<00:00,  2.47it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  16 89.0 489.75 489.75 1239 1239
100%|██████████| 1112/1112 [07:15<00:00,  2.55it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  17 94.0 546.98 546.98 1112 1112
100%|██████████| 1869/1869 [12:13<00:00,  2.55it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  18 100.0 369.25 369.25 1869 1869
```

__Currently__: ordering for the current fine tuning process:
1. train tokenizer and fit model to GUM
2. process NewsTweet documents to integrate sparse post-training statistics (requires mr implementation and updates to the vocabularies/indices)
3. update the ife and dense model, i.e., produce new statistics and dimensionalities
4. fine tune output heads to GUM, and _combine_ them with the dense model from (3), i.e., don't just replace as is current.

__Preliminarily__: this does seem to present performance benefits, but as is usual will require 'big data' statistics to become competitive. In particular, the (tokenization, least of all), counting, sorting, and aggregation of co-occurrence counts must all be distributed for the statistical resolution required to approach performance gains aking to more-advanced systems. Currently, a spark-based MR system is implemented for these (all but tokenization).

- small/ife/sparse/wt/c-encoded/hr-bpe-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00,  4.88it/s]
Evaluating language model..
100%|██████████| 1915/1915 [00:25<00:00, 74.43it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 223.64 3238.49 1915 1201
```

- small/ife/sparse/wt/c-encoded/hr-bpe-tokenized/space=False

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00,  4.33it/s]
Evaluating language model..
100%|██████████| 1201/1201 [00:22<00:00, 53.78it/s] 
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 1235.43 1235.43 1201 1201
```

- small/ife/sparse/wt/c-encoded/space-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00, 3021.83it/s]
Evaluating language model..
100%|██████████| 1447/1447 [00:10<00:00, 131.96it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 40.57 1416.55 1447 733
```

- full/ife/sparse/wt/c-encoded/hr-bpe-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 62/62 [00:53<00:00,  1.16it/s]
Evaluating language model..
100%|██████████| 1696/1696 [24:30<00:00,  1.15it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.49 226.6 1696 982
100%|██████████| 10440/10440 [2:39:05<00:00,  1.09it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 35.27 340.94 10440 5984
```

- full/ife/sparse/wt/c-encoded/space-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 62/62 [00:00<00:00, 976.10it/s]
Evaluating language model..
100%|██████████| 1447/1447 [34:20<00:00,  1.42s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.28 671.17 1447 733
```

- full/ife/sparse/wt/c-encoded/space-tokenized/space=False

```
Tokenizing documents...
100%|██████████| 62/62 [00:54<00:00,  1.13it/s]
Evaluating language model..
100%|██████████| 1696/1696 [26:18<00:00,  1.07it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 28.34 251.46 1696 982
100%|██████████| 10440/10440 [3:16:04<00:00,  1.13s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  2 3.0 36.6 366.0 10440 5984
100%|██████████| 4428/4428 [1:05:40<00:00,  1.12it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  3 5.0 33.68 341.82 4428 2487
100%|██████████| 15789/15789 [4:52:55<00:00,  1.11s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  4 6.0 33.26 294.04 15789 9118
100%|██████████| 4020/4020 [1:57:27<00:00,  1.75s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  5 8.0 32.2 354.75 4020 2263
```

- small/ife/sparse/wt/c-encoded/hr-bpe-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00,  4.94it/s]
Evaluating language model..
100%|██████████| 1915/1915 [04:31<00:00,  7.06it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 209.37 2907.44 1915 1201
```

- small/ife/sparse/wt/c-encoded/hr-bpe-tokenized/space=False

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00,  6.44it/s]
Evaluating language model..
100%|██████████| 1201/1201 [01:39<00:00, 12.05it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 1171.07 1171.07 1201 1201
```

- small/ife/sparse/wt/c-encoded/space-tokenized/space=True

```
Tokenizing documents...
100%|██████████| 1/1 [00:00<00:00, 2755.78it/s]
Evaluating language model..
100%|██████████| 1447/1447 [02:18<00:00, 10.47it/s]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 100.0 35.54 1089.28 1447 733
```

- full/ife/sparse/wt/c-encoded/space-tokenized/space=True
```
Tokenizing documents...
100%|██████████| 62/62 [00:00<00:00, 1052.76it/s]
Evaluating language model..
100%|██████████| 1447/1447 [46:43<00:00,  1.94s/it]
document no., pct. complete, 1/<P>, 1/<P> (nsp), M, M (nsp):  1 2.0 27.31 672.87 1447 733
```

The below models are summarized as:
```
- Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 5787912 10704 485 10185 12054 177 5.309
- Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 13442208 10704 485 10185 12054 177 12.33
- Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 15964670 10704 485 10185 12054 177 14.644
- Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 20152225 10704 485 10185 12054 177 18.485
```

- With IFE, trained on 132 GUM train documents, then stenciled on all 18 GUM test documents:

```
Tokenizing documents...
100%|██████████| 18/18 [00:02<00:00,  8.59it/s]
Evaluating language model..
100%|██████████| 2664/2664 [10:32<00:00,  4.21it/s]
document no., pct. complete, 1/<P>, M:  1 6.0 435.0 2664
100%|██████████| 2540/2540 [09:58<00:00,  4.25it/s]
document no., pct. complete, 1/<P>, M:  2 11.0 2654.38 2540
100%|██████████| 1869/1869 [07:27<00:00,  4.18it/s]
document no., pct. complete, 1/<P>, M:  3 17.0 1822.74 1869
100%|██████████| 2596/2596 [10:08<00:00,  4.26it/s]
document no., pct. complete, 1/<P>, M:  4 22.0 415.06 2596
100%|██████████| 2014/2014 [08:07<00:00,  4.13it/s]
document no., pct. complete, 1/<P>, M:  5 28.000000000000004 81.67 2014
100%|██████████| 2610/2610 [10:12<00:00,  4.26it/s]
document no., pct. complete, 1/<P>, M:  6 33.0 579.85 2610
100%|██████████| 2894/2894 [11:31<00:00,  4.18it/s]
document no., pct. complete, 1/<P>, M:  7 39.0 180.72 2894
100%|██████████| 2099/2099 [08:02<00:00,  4.35it/s]
document no., pct. complete, 1/<P>, M:  8 44.0 369.72 2099
100%|██████████| 2299/2299 [08:56<00:00,  4.28it/s]
document no., pct. complete, 1/<P>, M:  9 50.0 825.7 2299
100%|██████████| 3463/3463 [13:15<00:00,  4.35it/s]
document no., pct. complete, 1/<P>, M:  10 56.00000000000001 745.92 3463
100%|██████████| 1656/1656 [06:28<00:00,  4.26it/s]
document no., pct. complete, 1/<P>, M:  11 61.0 649.48 1656
100%|██████████| 2576/2576 [09:50<00:00,  4.36it/s]
document no., pct. complete, 1/<P>, M:  12 67.0 263.56 2576
100%|██████████| 2726/2726 [10:35<00:00,  4.29it/s]
document no., pct. complete, 1/<P>, M:  13 72.0 285.6 2726
100%|██████████| 2154/2154 [08:13<00:00,  4.36it/s]
document no., pct. complete, 1/<P>, M:  14 78.0 75.1 2154
100%|██████████| 2945/2945 [11:26<00:00,  4.29it/s]
document no., pct. complete, 1/<P>, M:  15 83.0 328.98 2945
100%|██████████| 1804/1804 [06:55<00:00,  4.35it/s]
document no., pct. complete, 1/<P>, M:  16 89.0 324.12 1804
100%|██████████| 1683/1683 [06:36<00:00,  4.24it/s]
document no., pct. complete, 1/<P>, M:  17 94.0 251.48 1683
100%|██████████| 2749/2749 [10:31<00:00,  4.35it/s]
document no., pct. complete, 1/<P>, M:  18 100.0 150.55 2749
```

- With IFE, trained on 132 GUM train documents, then stenciled on all 18 GUM test documents, rhyme = 0.5:

```
Tokenizing documents...
100%|██████████| 18/18 [00:02<00:00,  8.82it/s]
Evaluating language model..
100%|██████████| 2664/2664 [35:46<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  1 6.0 434.03 2664
100%|██████████| 2540/2540 [34:12<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  2 11.0 1928.54 2540
100%|██████████| 1869/1869 [25:10<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  3 17.0 1430.62 1869
100%|██████████| 2596/2596 [35:04<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  4 22.0 377.13 2596
100%|██████████| 2014/2014 [27:12<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  5 28.000000000000004 79.53 2014
100%|██████████| 2610/2610 [35:01<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  6 33.0 500.4 2610
100%|██████████| 2894/2894 [39:02<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  7 39.0 170.84 2894
100%|██████████| 2099/2099 [28:22<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  8 44.0 350.78 2099
100%|██████████| 2299/2299 [31:00<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  9 50.0 705.15 2299
100%|██████████| 3463/3463 [46:42<00:00,  1.24it/s]
document no., pct. complete, 1/<P>, M:  10 56.00000000000001 641.26 3463
100%|██████████| 1656/1656 [22:31<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  11 61.0 569.62 1656
100%|██████████| 2576/2576 [34:55<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  12 67.0 255.25 2576
100%|██████████| 2726/2726 [36:55<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  13 72.0 266.36 2726
100%|██████████| 2154/2154 [29:07<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  14 78.0 76.19 2154
100%|██████████| 2945/2945 [40:08<00:00,  1.22it/s]
document no., pct. complete, 1/<P>, M:  15 83.0 309.69 2945
100%|██████████| 1804/1804 [24:29<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  16 89.0 278.59 1804
100%|██████████| 1683/1683 [22:50<00:00,  1.23it/s]
document no., pct. complete, 1/<P>, M:  17 94.0 243.09 1683
100%|██████████| 2749/2749 [37:24<00:00,  1.22it/s]
document no., pct. complete, 1/<P>, M:  18 100.0 143.83 2749
```

- With IFE, trained on 132 GUM train documents, then 5,000 NewsTweet Pre/Post-training articles, then stenciled on all 18 GUM test documents:

```
Tokenizing documents..
100%|██████████| 18/18 [00:02<00:00,  8.80it/s]
Evaluating language model..
100%|██████████| 2664/2664 [07:23<00:00,  6.01it/s]
document no., pct. complete, 1/<P>, M:  1 6.0 101.64 2664
100%|██████████| 2540/2540 [06:57<00:00,  6.08it/s]
document no., pct. complete, 1/<P>, M:  2 11.0 1224.9 2540
100%|██████████| 1869/1869 [05:16<00:00,  5.91it/s]
document no., pct. complete, 1/<P>, M:  3 17.0 869.15 1869
100%|██████████| 2596/2596 [07:07<00:00,  6.07it/s]
document no., pct. complete, 1/<P>, M:  4 22.0 198.22 2596
100%|██████████| 2014/2014 [05:30<00:00,  6.09it/s]
document no., pct. complete, 1/<P>, M:  5 28.000000000000004 59.24 2014
100%|██████████| 2610/2610 [07:21<00:00,  5.92it/s]
document no., pct. complete, 1/<P>, M:  6 33.0 218.71 2610
100%|██████████| 2894/2894 [07:56<00:00,  6.08it/s]
document no., pct. complete, 1/<P>, M:  7 39.0 82.13 2894
100%|██████████| 2099/2099 [05:48<00:00,  6.03it/s]
document no., pct. complete, 1/<P>, M:  8 44.0 99.2 2099
100%|██████████| 2299/2299 [06:30<00:00,  5.89it/s]
document no., pct. complete, 1/<P>, M:  9 50.0 280.71 2299
100%|██████████| 3463/3463 [09:31<00:00,  6.06it/s]
document no., pct. complete, 1/<P>, M:  10 56.00000000000001 164.21 3463
100%|██████████| 1656/1656 [04:49<00:00,  5.72it/s]
document no., pct. complete, 1/<P>, M:  11 61.0 194.34 1656
100%|██████████| 2576/2576 [08:26<00:00,  5.09it/s]
document no., pct. complete, 1/<P>, M:  12 67.0 89.3 2576
100%|██████████| 2726/2726 [07:50<00:00,  5.80it/s]
document no., pct. complete, 1/<P>, M:  13 72.0 147.42 2726
100%|██████████| 2154/2154 [06:19<00:00,  5.67it/s]
document no., pct. complete, 1/<P>, M:  14 78.0 38.17 2154
100%|██████████| 2945/2945 [10:30<00:00,  4.67it/s]
document no., pct. complete, 1/<P>, M:  15 83.0 117.08 2945
100%|██████████| 1804/1804 [12:28<00:00,  2.41it/s]
document no., pct. complete, 1/<P>, M:  16 89.0 142.28 1804
100%|██████████| 1683/1683 [06:10<00:00,  4.54it/s]
document no., pct. complete, 1/<P>, M:  17 94.0 100.67 1683
100%|██████████| 2749/2749 [08:16<00:00,  5.54it/s]
document no., pct. complete, 1/<P>, M:  18 100.0 78.67 2749
```

- With IFE, trained on 132 GUM train documents, then 10,000 NewsTweet Pre/Post-training articles, then stenciled on all 18 GUM test documents:

```
Tokenizing documents..
100%|██████████| 18/18 [00:02<00:00,  8.85it/s]
Evaluating language model..
100%|██████████| 2664/2664 [05:43<00:00,  7.75it/s]
document no., pct. complete, 1/<P>, M:  1 6.0 153.41 2664
100%|██████████| 2540/2540 [05:41<00:00,  7.44it/s]
document no., pct. complete, 1/<P>, M:  2 11.0 1324.68 2540
100%|██████████| 1869/1869 [04:02<00:00,  7.70it/s]
document no., pct. complete, 1/<P>, M:  3 17.0 977.94 1869
100%|██████████| 2596/2596 [05:36<00:00,  7.70it/s]
document no., pct. complete, 1/<P>, M:  4 22.0 224.7 2596
100%|██████████| 2014/2014 [04:34<00:00,  7.34it/s]
document no., pct. complete, 1/<P>, M:  5 28.000000000000004 60.83 2014
100%|██████████| 2610/2610 [05:38<00:00,  7.71it/s]
document no., pct. complete, 1/<P>, M:  6 33.0 270.45 2610
100%|██████████| 2894/2894 [06:15<00:00,  7.70it/s]
document no., pct. complete, 1/<P>, M:  7 39.0 98.84 2894
100%|██████████| 2099/2099 [04:33<00:00,  7.67it/s]
document no., pct. complete, 1/<P>, M:  8 44.0 121.84 2099
100%|██████████| 2299/2299 [05:13<00:00,  7.33it/s]
document no., pct. complete, 1/<P>, M:  9 50.0 343.57 2299
100%|██████████| 3463/3463 [07:31<00:00,  7.67it/s]
document no., pct. complete, 1/<P>, M:  10 56.00000000000001 239.94 3463
100%|██████████| 1656/1656 [03:36<00:00,  7.66it/s]
document no., pct. complete, 1/<P>, M:  11 61.0 251.67 1656
100%|██████████| 2576/2576 [05:48<00:00,  7.39it/s]
document no., pct. complete, 1/<P>, M:  12 67.0 114.0 2576
100%|██████████| 2726/2726 [05:56<00:00,  7.64it/s]
document no., pct. complete, 1/<P>, M:  13 72.0 171.13 2726
100%|██████████| 2154/2154 [04:40<00:00,  7.69it/s]
document no., pct. complete, 1/<P>, M:  14 78.0 44.69 2154
100%|██████████| 2945/2945 [06:38<00:00,  7.39it/s]
document no., pct. complete, 1/<P>, M:  15 83.0 151.02 2945
100%|██████████| 1804/1804 [03:55<00:00,  7.65it/s]
document no., pct. complete, 1/<P>, M:  16 89.0 160.74 1804
100%|██████████| 1683/1683 [03:40<00:00,  7.63it/s]
document no., pct. complete, 1/<P>, M:  17 94.0 118.86 1683
100%|██████████| 2749/2749 [05:59<00:00,  7.65it/s]
document no., pct. complete, 1/<P>, M:  18 100.0 90.08 2749
```

- With IFE, trained on 132 GUM train documents, then 14,198 (all) NewsTweet Pre/Post-training articles, then stenciled on all 18 GUM test documents:

```
Tokenizing documents..
100%|██████████| 18/18 [00:02<00:00,  8.94it/s]
Evaluating language model..
100%|██████████| 2664/2664 [07:51<00:00,  5.65it/s]
document no., pct. complete, 1/<P>, M:  1 6.0 87.5 2664
100%|██████████| 2540/2540 [08:46<00:00,  4.82it/s]
document no., pct. complete, 1/<P>, M:  2 11.0 1246.43 2540
100%|██████████| 1869/1869 [06:26<00:00,  4.84it/s]
document no., pct. complete, 1/<P>, M:  3 17.0 907.98 1869
100%|██████████| 2596/2596 [09:09<00:00,  4.72it/s]
document no., pct. complete, 1/<P>, M:  4 22.0 185.52 2596
100%|██████████| 2014/2014 [06:57<00:00,  4.82it/s]
document no., pct. complete, 1/<P>, M:  5 28.000000000000004 60.31 2014
100%|██████████| 2610/2610 [09:11<00:00,  4.73it/s]
document no., pct. complete, 1/<P>, M:  6 33.0 211.75 2610
100%|██████████| 2894/2894 [10:00<00:00,  4.82it/s]
document no., pct. complete, 1/<P>, M:  7 39.0 77.63 2894
100%|██████████| 2099/2099 [07:28<00:00,  4.68it/s]
document no., pct. complete, 1/<P>, M:  8 44.0 94.97 2099
100%|██████████| 2299/2299 [07:57<00:00,  4.81it/s]
document no., pct. complete, 1/<P>, M:  9 50.0 275.11 2299
100%|██████████| 3463/3463 [12:12<00:00,  4.73it/s]
document no., pct. complete, 1/<P>, M:  10 56.00000000000001 153.55 3463
100%|██████████| 1656/1656 [05:43<00:00,  4.81it/s]
document no., pct. complete, 1/<P>, M:  11 61.0 181.12 1656
100%|██████████| 2576/2576 [08:58<00:00,  4.78it/s]
document no., pct. complete, 1/<P>, M:  12 67.0 79.46 2576
100%|██████████| 2726/2726 [09:38<00:00,  4.71it/s]
document no., pct. complete, 1/<P>, M:  13 72.0 134.32 2726
100%|██████████| 2154/2154 [07:31<00:00,  4.77it/s]
document no., pct. complete, 1/<P>, M:  14 78.0 36.48 2154
100%|██████████| 2945/2945 [10:24<00:00,  4.71it/s]
document no., pct. complete, 1/<P>, M:  15 83.0 105.72 2945
100%|██████████| 1804/1804 [06:16<00:00,  4.79it/s]
document no., pct. complete, 1/<P>, M:  16 89.0 138.55 1804
100%|██████████| 1683/1683 [05:59<00:00,  4.68it/s]
document no., pct. complete, 1/<P>, M:  17 94.0 95.61 1683
100%|██████████| 2749/2749 [09:35<00:00,  4.78it/s]
document no., pct. complete, 1/<P>, M:  18 100.0 76.1 2749
```

In [ ]:
model.generate(m = 50, seed = seed)